<a href="https://colab.research.google.com/github/Sandsuraj/code-project/blob/main/Copy_of_INTERNSHIP_SUBMISSION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!pip install -q transformers torch librosa pydub nltk sentencepiece accelerate
!pip install -q git+https://github.com/openai/whisper.git


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 6.4 MB/s eta 0:00:00


In [ ]:
import whisper
import librosa
import nltk
import torch
import re
from transformers import pipeline
from nltk.tokenize import sent_tokenize


In [ ]:
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
def preprocess_audio(audio_path):
    audio, sr = librosa.load(audio_path, sr=16000)
    return audio


In [ ]:
asr_model = whisper.load_model("base")  # base = good accuracy + fast


100%|███████████████████████████████████████| 139M/139M [00:02<00:00, 64.5MiB/s]


In [ ]:
def transcribe_audio(audio_path):
    result = asr_model.transcribe(audio_path)
    return result["text"]


In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\b(uh|um|ah|er|mm)\b', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()


In [ ]:
def chunk_text(text, max_words=500):
    sentences = sent_tokenize(text)
    chunks = []
    current_chunk = []

    word_count = 0
    for sentence in sentences:
        words = sentence.split()
        word_count += len(words)
        current_chunk.append(sentence)

        if word_count >= max_words:
            chunks.append(" ".join(current_chunk))
            current_chunk = []
            word_count = 0

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks


In [ ]:
summarizer = pipeline(
    "summarization",
    model="facebook/bart-large-cnn",
    device=0 if torch.cuda.is_available() else -1
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [ ]:
def summarize_chunks(chunks):
    summaries = []

    for chunk in chunks:
        summary = summarizer(
            chunk,
            max_length=150,
            min_length=50,
            do_sample=False
        )[0]['summary_text']

        summaries.append(summary)

    return summaries


In [ ]:
def final_summary(chunk_summaries):
    combined_text = " ".join(chunk_summaries)

    final = summarizer(
        combined_text,
        max_length=200,
        min_length=80,
        do_sample=False
    )[0]['summary_text']

    return final


In [ ]:
def podcast_summarizer(audio_path):
    print("🔊 Transcribing audio...")
    transcript = transcribe_audio(audio_path)

    print("🧹 Cleaning transcript...")
    clean_transcript = clean_text(transcript)

    print("✂️ Chunking text...")
    chunks = chunk_text(clean_transcript)

    print(f"📦 Total Chunks: {len(chunks)}")

    print("🧠 Summarizing chunks...")
    chunk_summaries = summarize_chunks(chunks)

    print("✨ Creating final summary...")
    final = final_summary(chunk_summaries)

    return {
        "transcript": clean_transcript,
        "chunk_summaries": chunk_summaries,
        "final_summary": final
    }


### Downloading a sample audio file

I'll download a sample audio file to demonstrate the `podcast_summarizer` function. If you have your own audio file, you can replace this with your file's path.

In [ ]:
# Download a sample audio file
!wget -O sample_audio.mp3 https://www.soundhelix.com/examples/mp3/SoundHelix-Song-1.mp3

audio_file_path = 'sample_audio.mp3'

print(f"Using audio file: {audio_file_path}")

--2026-01-19 15:46:50--  https://www.soundhelix.com/examples/mp3/SoundHelix-Song-1.mp3
Resolving www.soundhelix.com (www.soundhelix.com)... 81.169.145.157, 2a01:238:20a:202:1157::
Connecting to www.soundhelix.com (www.soundhelix.com)|81.169.145.157|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8945229 (8.5M) [audio/mpeg]
Saving to: ‘sample_audio.mp3’

sample_audio.mp3    100%[===================>]   8.53M  7.63MB/s    in 1.1s    

2026-01-19 15:46:52 (7.63 MB/s) - ‘sample_audio.mp3’ saved [8945229/8945229]

Using audio file: sample_audio.mp3


### Calling the `podcast_summarizer` function

In [ ]:
# Call the podcast_summarizer function again, now that NLTK resources are ready
summary_output = podcast_summarizer(audio_file_path)

print("\nFINAL PODCAST SUMMARY:")
print(summary_output["final_summary"])

🔊 Transcribing audio...


/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
Your max_length is set to 150, but your input_length is only 4. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=2)


🧹 Cleaning transcript...
✂️ Chunking text...
📦 Total Chunks: 1
🧠 Summarizing chunks...


Your max_length is set to 200, but your input_length is only 57. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=28)


✨ Creating final summary...

FINAL PODCAST SUMMARY:
Ortega de Orellana is one of the world's most successful soap operas. It is based in Madrid, Spain, and is known as the Ortega d'Orellana. The ortega has been in business for more than 100 years and is now in its 100th year of existence. The soap opera is currently being shown on Channel 4 in the UK.


In [ ]:
!pip install nltk

In [ ]:
audio_file_path = 'sample_audio.mp3'
print(f"The audio file path is: {audio_file_path}")

# Example of how you might open and read an audio file (this is illustrative and might need specific libraries for audio):
# with open(audio_file_path, 'rb') as f:
#     # Read some content or pass to an audio processing library
#     print(f"Successfully opened {audio_file_path}")

The audio file path is: sample_audio.mp3


In [ ]:
df.to_csv("/mnt/data/podcast_summary_results.csv", index=False)


OSError: Cannot save file into a non-existent directory: '/mnt/data'

In [ ]:
import pandas as pd

# Convert the summary_output dictionary to a DataFrame
# Since chunk_summaries is a list, we'll store it as a list in the DataFrame cell
df = pd.DataFrame({
    'transcript': [summary_output['transcript']],
    'chunk_summaries': [summary_output['chunk_summaries']],
    'final_summary': [summary_output['final_summary']]
})

# Display the DataFrame
display(df)

In [ ]:
print("FINAL PODCAST SUMMARY:")
print(df['final_summary'].iloc[0])

FINAL PODCAST SUMMARY:
iReport.com will feature iReporter photos in a weekly Travel Snapshots gallery. Please submit your best shots of the U.S. for next week. Visit CNN.com/Travel each week for a new gallery of snapshots from around the world. Osh Osh, Osh. osh. iReporters: Share your best photos of the United States with CNN iReport at iReport.


In [ ]:
df.to_csv('podcast_summary_results.csv', index=False)
print("DataFrame 'df' saved to 'podcast_summary_results.csv'")

In [ ]:
import pandas as pd

df = pd.read_csv("/content/podcast_summary_results.csv")


In [ ]:
df.columns


In [ ]:
Index(['Podcast_Name', 'Episode_Title', 'Episode_Length_minutes',
       'Genre', 'Host_Popularity_percentage', 'Publication_Day',
       'Publication_Time', 'Guest_Popularity_percentage',
       'Number_of_Ads', 'Episode_Sentiment', 'Listening_Time_minutes'],
      dtype='object')



In [ ]:
# Generate sample data for 'chunk_summaries'
sample_chunk_summaries = [
    "This chunk discusses technology and AI innovations.",
    "Another chunk focuses on machine learning's impact on daily life.",
    "The final chunk considers ethical implications of AI deployment."
]

# Update the 'chunk_summaries' column in the DataFrame
df.at[0, 'chunk_summaries'] = sample_chunk_summaries

print("DataFrame 'df' updated with sample 'chunk_summaries' data.")
display(df)

In [ ]:
# Generate a sample transcript
sample_transcript = "This is a sample podcast transcript. It contains various sentences about technology, artificial intelligence, and future innovations. We discuss new developments in machine learning and their impact on daily life. The conversation also touches upon ethical considerations in AI deployment."

# Update the 'transcript' column in the DataFrame
# Assuming df has at least one row, we update the first row's transcript
df.at[0, 'transcript'] = sample_transcript

print("DataFrame 'df' updated with sample transcript data.")
display(df)

In [ ]:
display(df.head())

In [ ]:
display(df.describe())

In [ ]:
display(df.head())

In [ ]:
display(df.head())

In [ ]:
print("Missing values in DataFrame df:")
print(df.isnull().sum())

In [ ]:
print(df.columns)

In [ ]:
print(df.columns)

In [ ]:
df[['Podcast_Name', 'Episode_Title']].head(10)


In [ ]:
print("DataFrame Info:")
df.info()

In [ ]:
df['transcript_word_count'] = df['transcript'].apply(lambda x: len(str(x).split()))
print("Descriptive statistics for transcript word count:")
display(df['transcript_word_count'].describe())

Descriptive statistics for transcript word count:


,transcript_word_count
count,1.0
mean,40.0
std,NaN
min,40.0
25%,40.0
50%,40.0
75%,40.0
max,40.0


In [ ]:
print("\nFirst 5 rows of DataFrame:")
display(df.head())


First 5 rows of DataFrame:


,transcript,chunk_summaries,final_summary,Podcast_Name,Episode_Title,transcript_word_count
0,This is a sample podcast transcript. It contai...,[This chunk discusses technology and AI innova...,iReport.com will feature iReporter photos in a...,The Tech Daily,AI in Focus: Ethical Implications,40


In [ ]:
import pandas as pd

df = pd.read_csv("/content/podcast_summary_results.csv")

In [ ]:
# Generate sample data for 'chunk_summaries'
sample_chunk_summaries = [
    "This chunk discusses technology and AI innovations.",
    "Another chunk focuses on machine learning's impact on daily life.",
    "The final chunk considers ethical implications of AI deployment."
]

# Update the 'chunk_summaries' column in the DataFrame
df.at[0, 'chunk_summaries'] = sample_chunk_summaries

print("DataFrame 'df' updated with sample 'chunk_summaries' data.")
display(df)

DataFrame 'df' updated with sample 'chunk_summaries' data.


,transcript,chunk_summaries,final_summary
0,osh,[This chunk discusses technology and AI innova...,iReport.com will feature iReporter photos in a...


In [ ]:
# Generate a sample transcript
sample_transcript = "This is a sample podcast transcript. It contains various sentences about technology, artificial intelligence, and future innovations. We discuss new developments in machine learning and their impact on daily life. The conversation also touches upon ethical considerations in AI deployment."

# Update the 'transcript' column in the DataFrame
# Assuming df has at least one row, we update the first row's transcript
df.at[0, 'transcript'] = sample_transcript

print("DataFrame 'df' updated with sample transcript data.")
display(df)

DataFrame 'df' updated with sample transcript data.


,transcript,chunk_summaries,final_summary
0,This is a sample podcast transcript. It contai...,[This chunk discusses technology and AI innova...,iReport.com will feature iReporter photos in a...


In [ ]:
import pandas as pd
import numpy as np

# Generate more varied sample data for transcripts and summaries
num_samples = 10

sample_transcripts = [
    "This is a sample podcast transcript about technology and artificial intelligence. We discuss machine learning, ethical considerations, and future innovations in great detail.",
    "A shorter transcript focusing on daily news and current events. We cover politics, economics, and social trends briefly.",
    "An in-depth discussion on space exploration, including recent missions to Mars and the future of human colonization. Long-form content with many scientific terms.",
    "A quick update on sports results from yesterday. Football, basketball, and tennis highlights are mentioned.",
    "This podcast explores historical events, specifically the causes and effects of World War II. It's a comprehensive overview lasting over an hour.",
    "A conversational piece about cooking and new recipes. Today's topic: baking sourdough bread from scratch. Detailed instructions and tips are provided.",
    "Interview with a successful entrepreneur discussing startup challenges and growth strategies. Inspiring stories and business advice.",
    "Meditation guide for beginners. Focus on breathing techniques and mindfulness exercises for stress reduction and mental clarity. Very calm and slow-paced.",
    "Review of the latest blockbuster movie releases. Spoilers ahead! Analysis of plot, acting, and special effects. A bit lengthy due to detailed critiques.",
    "Learning a new language, specifically Spanish. Basic phrases, grammar rules, and cultural insights. A moderate length episode for intermediate learners."
]

sample_chunk_summaries_list = [
    ["Tech and AI discussed.", "ML impact on daily life.", "Ethical AI deployment."],
    ["Daily news, politics, economy, social trends."],
    ["Space exploration, Mars missions, human colonization.", "Scientific terms in depth."],
    ["Yesterday's sports results, football, basketball, tennis."],
    ["Historical events, WWII causes, comprehensive overview."],
    ["Cooking, sourdough baking, detailed instructions."],
    ["Entrepreneur interview, startup challenges, growth strategies."],
    ["Meditation for beginners, breathing, mindfulness."],
    ["Movie reviews, spoilers, plot analysis, acting, effects.", "Lengthy critiques."],
    ["Spanish language learning, phrases, grammar, culture."]
]

sample_final_summaries = [
    "This episode delves into technology and AI innovations, covering machine learning's societal impact and ethical considerations in deployment.",
    "A concise summary of daily news, touching upon recent developments in politics, economics, and general social trends.",
    "Explore space missions to Mars and the future of human colonization in this scientific, long-form podcast.",
    "Get a quick recap of yesterday's sports, including highlights from football, basketball, and tennis matches.",
    "A deep dive into World War II, examining its historical causes and lasting global effects.",
    "Learn to bake sourdough bread from scratch with detailed instructions and expert tips in this culinary episode.",
    "An inspiring interview with an entrepreneur sharing insights on startup challenges and effective growth strategies.",
    "A beginner-friendly guide to meditation, focusing on breathing and mindfulness for stress reduction.",
    "Detailed analysis and critiques of recent blockbuster movie releases, covering plot, acting, and special effects, with spoilers.",
    "A language learning podcast offering basic Spanish phrases, grammar rules, and cultural insights for intermediate learners."
]

sample_podcast_names = [
    'The Tech Talk',
    'Daily Briefing',
    'Cosmic Frontiers',
    'Sports Recap',
    'History Uncovered',
    'The Cooking Channel',
    'Startup Stories',
    'Mindful Moments',
    'Cinema Scope',
    'Language Learners'
]

sample_episode_titles = [
    'AI and Ethics in 2026',
    'Today\'s Headlines',
    'Journey to the Red Planet',
    'Weekend Sports Highlights',
    'World War II: A Deep Dive',
    'Sourdough Secrets',
    'From Idea to Empire',
    'Beginner\'s Meditation',
    'New Movie Releases',
    'Spanish for Travelers'
]

# Create a new DataFrame with the generated sample data
df = pd.DataFrame({
    'transcript': sample_transcripts,
    'chunk_summaries': sample_chunk_summaries_list,
    'final_summary': sample_final_summaries,
    'Podcast_Name': sample_podcast_names,
    'Episode_Title': sample_episode_titles
})

print("DataFrame 'df' updated with more sample data.")
display(df.head())

DataFrame 'df' updated with more sample data.


,transcript,chunk_summaries,final_summary,Podcast_Name,Episode_Title
0,This is a sample podcast transcript about tech...,"[Tech and AI discussed., ML impact on daily li...",This episode delves into technology and AI inn...,The Tech Talk,AI and Ethics in 2026
1,A shorter transcript focusing on daily news an...,"[Daily news, politics, economy, social trends.]","A concise summary of daily news, touching upon...",Daily Briefing,Today's Headlines
2,"An in-depth discussion on space exploration, i...","[Space exploration, Mars missions, human colon...",Explore space missions to Mars and the future ...,Cosmic Frontiers,Journey to the Red Planet
3,A quick update on sports results from yesterda...,"[Yesterday's sports results, football, basketb...","Get a quick recap of yesterday's sports, inclu...",Sports Recap,Weekend Sports Highlights
4,"This podcast explores historical events, speci...","[Historical events, WWII causes, comprehensive...","A deep dive into World War II, examining its h...",History Uncovered,World War II: A Deep Dive


In [ ]:
df['transcript_word_count'] = df['transcript'].apply(lambda x: len(str(x).split()))
print("Descriptive statistics for transcript word count with new data:")
display(df['transcript_word_count'].describe())

Descriptive statistics for transcript word count with new data:


,transcript_word_count
count,10.000000
mean,20.200000
std,2.936362
min,15.000000
25%,18.500000
50%,21.000000
75%,22.750000
max,23.000000


In [ ]:
# Add 'Podcast_Name' and 'Episode_Title' columns with sample data
df['Podcast_Name'] = 'The Tech Daily'
df['Episode_Title'] = 'AI in Focus: Ethical Implications'

print("DataFrame 'df' updated with 'Podcast_Name' and 'Episode_Title' columns.")
display(df)

DataFrame 'df' updated with 'Podcast_Name' and 'Episode_Title' columns.


,transcript,chunk_summaries,final_summary,Podcast_Name,Episode_Title,transcript_word_count
0,This is a sample podcast transcript about tech...,"[Tech and AI discussed., ML impact on daily li...",This episode delves into technology and AI inn...,The Tech Daily,AI in Focus: Ethical Implications,23
1,A shorter transcript focusing on daily news an...,"[Daily news, politics, economy, social trends.]","A concise summary of daily news, touching upon...",The Tech Daily,AI in Focus: Ethical Implications,18
2,"An in-depth discussion on space exploration, i...","[Space exploration, Mars missions, human colon...",Explore space missions to Mars and the future ...,The Tech Daily,AI in Focus: Ethical Implications,23
3,A quick update on sports results from yesterda...,"[Yesterday's sports results, football, basketb...","Get a quick recap of yesterday's sports, inclu...",The Tech Daily,AI in Focus: Ethical Implications,15
4,"This podcast explores historical events, speci...","[Historical events, WWII causes, comprehensive...","A deep dive into World War II, examining its h...",The Tech Daily,AI in Focus: Ethical Implications,22
5,A conversational piece about cooking and new r...,"[Cooking, sourdough baking, detailed instructi...",Learn to bake sourdough bread from scratch wit...,The Tech Daily,AI in Focus: Ethical Implications,21
6,Interview with a successful entrepreneur discu...,"[Entrepreneur interview, startup challenges, g...",An inspiring interview with an entrepreneur sh...,The Tech Daily,AI in Focus: Ethical Implications,16
7,Meditation guide for beginners. Focus on breat...,"[Meditation for beginners, breathing, mindfuln...","A beginner-friendly guide to meditation, focus...",The Tech Daily,AI in Focus: Ethical Implications,21
8,Review of the latest blockbuster movie release...,"[Movie reviews, spoilers, plot analysis, actin...",Detailed analysis and critiques of recent bloc...,The Tech Daily,AI in Focus: Ethical Implications,23
9,"Learning a new language, specifically Spanish....","[Spanish language learning, phrases, grammar, ...",A language learning podcast offering basic Spa...,The Tech Daily,AI in Focus: Ethical Implications,20


In [ ]:
print("DataFrame Info:")
df.info()

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   transcript             10 non-null     object
 1   chunk_summaries        10 non-null     object
 2   final_summary          10 non-null     object
 3   Podcast_Name           10 non-null     object
 4   Episode_Title          10 non-null     object
 5   transcript_word_count  10 non-null     int64 
dtypes: int64(1), object(5)
memory usage: 612.0+ bytes


In [ ]:
print("\nFirst 5 rows of DataFrame:")
display(df.head())


First 5 rows of DataFrame:


,transcript,chunk_summaries,final_summary,Podcast_Name,Episode_Title,transcript_word_count
0,This is a sample podcast transcript about tech...,"[Tech and AI discussed., ML impact on daily li...",This episode delves into technology and AI inn...,The Tech Daily,AI in Focus: Ethical Implications,23
1,A shorter transcript focusing on daily news an...,"[Daily news, politics, economy, social trends.]","A concise summary of daily news, touching upon...",The Tech Daily,AI in Focus: Ethical Implications,18
2,"An in-depth discussion on space exploration, i...","[Space exploration, Mars missions, human colon...",Explore space missions to Mars and the future ...,The Tech Daily,AI in Focus: Ethical Implications,23
3,A quick update on sports results from yesterda...,"[Yesterday's sports results, football, basketb...","Get a quick recap of yesterday's sports, inclu...",The Tech Daily,AI in Focus: Ethical Implications,15
4,"This podcast explores historical events, speci...","[Historical events, WWII causes, comprehensive...","A deep dive into World War II, examining its h...",The Tech Daily,AI in Focus: Ethical Implications,22


In [ ]:
print("DataFrame Info:")
df.info()

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   transcript             10 non-null     object
 1   chunk_summaries        10 non-null     object
 2   final_summary          10 non-null     object
 3   Podcast_Name           10 non-null     object
 4   Episode_Title          10 non-null     object
 5   transcript_word_count  10 non-null     int64 
dtypes: int64(1), object(5)
memory usage: 612.0+ bytes


In [ ]:
import pandas as pd

# Convert the summary_output dictionary to a DataFrame
# Since chunk_summaries is a list, we'll store it as a list in the DataFrame cell
df = pd.DataFrame({
    'transcript': [summary_output['transcript']],
    'chunk_summaries': [summary_output['chunk_summaries']],
    'final_summary': [summary_output['final_summary']]
})

In [ ]:
df.to_csv('podcast_summary_results.csv', index=False)
print("DataFrame 'df' saved to 'podcast_summary_results.csv'")

DataFrame 'df' saved to 'podcast_summary_results.csv'


In [ ]:
import pandas as pd

df = pd.read_csv("/content/podcast_summary_results.csv")

In [ ]:
# Generate sample data for 'chunk_summaries'
sample_chunk_summaries = [
    "This chunk discusses technology and AI innovations.",
    "Another chunk focuses on machine learning's impact on daily life.",
    "The final chunk considers ethical implications of AI deployment."
]

# Update the 'chunk_summaries' column in the DataFrame
df.at[0, 'chunk_summaries'] = sample_chunk_summaries

print("DataFrame 'df' updated with sample 'chunk_summaries' data.")
display(df)

DataFrame 'df' updated with sample 'chunk_summaries' data.


,transcript,chunk_summaries,final_summary
0,orte,[This chunk discusses technology and AI innova...,Ortega de Orellana is one of the world's most ...


In [ ]:
# Generate a sample transcript
sample_transcript = "This is a sample podcast transcript. It contains various sentences about technology, artificial intelligence, and future innovations. We discuss new developments in machine learning and their impact on daily life. The conversation also touches upon ethical considerations in AI deployment."

# Update the 'transcript' column in the DataFrame
# Assuming df has at least one row, we update the first row's transcript
df.at[0, 'transcript'] = sample_transcript

print("DataFrame 'df' updated with sample transcript data.")
display(df)

DataFrame 'df' updated with sample transcript data.


,transcript,chunk_summaries,final_summary
0,This is a sample podcast transcript. It contai...,[This chunk discusses technology and AI innova...,Ortega de Orellana is one of the world's most ...


In [ ]:
# Add 'Podcast_Name' and 'Episode_Title' columns with sample data
df['Podcast_Name'] = 'The Tech Daily'
df['Episode_Title'] = 'AI in Focus: Ethical Implications'

print("DataFrame 'df' updated with 'Podcast_Name' and 'Episode_Title' columns.")
display(df)

DataFrame 'df' updated with 'Podcast_Name' and 'Episode_Title' columns.


,transcript,chunk_summaries,final_summary,Podcast_Name,Episode_Title
0,This is a sample podcast transcript. It contai...,[This chunk discusses technology and AI innova...,Ortega de Orellana is one of the world's most ...,The Tech Daily,AI in Focus: Ethical Implications


In [ ]:
print("DataFrame Info:")
df.info()

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   transcript       1 non-null      object
 1   chunk_summaries  1 non-null      object
 2   final_summary    1 non-null      object
 3   Podcast_Name     1 non-null      object
 4   Episode_Title    1 non-null      object
dtypes: object(5)
memory usage: 172.0+ bytes


In [ ]:
print("\nFirst 5 rows of DataFrame:")
display(df.head())


First 5 rows of DataFrame:


,transcript,chunk_summaries,final_summary,Podcast_Name,Episode_Title
0,This is a sample podcast transcript. It contai...,[This chunk discusses technology and AI innova...,Ortega de Orellana is one of the world's most ...,The Tech Daily,AI in Focus: Ethical Implications


In [ ]:
!pip install -q transformers torch librosa pydub nltk sentencepiece accelerate
!pip install -q git+https://github.com/openai/whisper.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import whisper
import librosa
import nltk
import torch
import re
from transformers import pipeline
from nltk.tokenize import sent_tokenize

In [ ]:
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
def preprocess_audio(audio_path):
    audio, sr = librosa.load(audio_path, sr=16000)
    return audio

In [ ]:
asr_model = whisper.load_model("base")

In [ ]:
def transcribe_audio(audio_path):
    result = asr_model.transcribe(audio_path)
    return result["text"]

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\b(uh|um|ah|er|mm)\b', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

In [ ]:
def chunk_text(text, max_words=500):
    sentences = sent_tokenize(text)
    chunks = []
    current_chunk = []

    word_count = 0
    for sentence in sentences:
        words = sentence.split()
        word_count += len(words)
        current_chunk.append(sentence)

        if word_count >= max_words:
            chunks.append(" ".join(current_chunk))
            current_chunk = []
            word_count = 0

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

In [ ]:
summarizer = pipeline(
    "summarization",
    model="facebook/bart-large-cnn",
    device=0 if torch.cuda.is_available() else -1
)

Device set to use cpu


In [ ]:
def summarize_chunks(chunks):
    summaries = []

    for chunk in chunks:
        summary = summarizer(
            chunk,
            max_length=150,
            min_length=50,
            do_sample=False
        )[0]['summary_text']

        summaries.append(summary)

    return summaries

In [ ]:
def final_summary(chunk_summaries):
    combined_text = " ".join(chunk_summaries)

    final = summarizer(
        combined_text,
        max_length=200,
        min_length=80,
        do_sample=False
    )[0]['summary_text']

    return final

In [ ]:
def podcast_summarizer(audio_path):
    print("🔊 Transcribing audio...")
    transcript = transcribe_audio(audio_path)

    print("🧹 Cleaning transcript...")
    clean_transcript = clean_text(transcript)

    print("✂️ Chunking text...")
    chunks = chunk_text(clean_transcript)

    print(f"📦 Total Chunks: {len(chunks)}")

    print("🧠 Summarizing chunks...")
    chunk_summaries = summarize_chunks(chunks)

    print("✨ Creating final summary...")
    final = final_summary(chunk_summaries)

    return {
        "transcript": clean_transcript,
        "chunk_summaries": chunk_summaries,
        "final_summary": final
    }

In [ ]:
# Download a sample audio file
!wget -O sample_audio.mp3 https://www.soundhelix.com/examples/mp3/SoundHelix-Song-1.mp3

audio_file_path = 'sample_audio.mp3'

print(f"Using audio file: {audio_file_path}")

--2026-01-20 17:29:11--  https://www.soundhelix.com/examples/mp3/SoundHelix-Song-1.mp3
Resolving www.soundhelix.com (www.soundhelix.com)... 81.169.145.157, 2a01:238:20a:202:1157::
Connecting to www.soundhelix.com (www.soundhelix.com)|81.169.145.157|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8945229 (8.5M) [audio/mpeg]
Saving to: ‘sample_audio.mp3’

sample_audio.mp3    100%[===================>]   8.53M  10.1MB/s    in 0.8s    

2026-01-20 17:29:12 (10.1 MB/s) - ‘sample_audio.mp3’ saved [8945229/8945229]

Using audio file: sample_audio.mp3


In [ ]:
# Call the podcast_summarizer function again, now that NLTK resources are ready
summary_output = podcast_summarizer(audio_file_path)

print("\nFINAL PODCAST SUMMARY:")
print(summary_output["final_summary"])

🔊 Transcribing audio...


/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
Your max_length is set to 150, but your input_length is only 6. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=3)


🧹 Cleaning transcript...
✂️ Chunking text...
📦 Total Chunks: 1
🧠 Summarizing chunks...


Your max_length is set to 200, but your input_length is only 60. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=30)


✨ Creating final summary...

FINAL PODCAST SUMMARY:
CNN.com will feature iReporter photos in a weekly Travel Snapshots gallery. Please submit your best shots for next week's gallery. Visit CNN.com/Travel each week for a new gallery of snapshots from around the world. For the latest from CNN, visit our Travel page. Back to the page you came from. Click here for the latest iReporters' photos from the world of travel.


In [ ]:
import pandas as pd

# Convert the summary_output dictionary to a DataFrame
# Since chunk_summaries is a list, we'll store it as a list in the DataFrame cell
df = pd.DataFrame({
    'transcript': [summary_output['transcript']],
    'chunk_summaries': [summary_output['chunk_summaries']],
    'final_summary': [summary_output['final_summary']]
})

In [ ]:
df.to_csv('podcast_summary_results.csv', index=False)
print("DataFrame 'df' saved to 'podcast_summary_results.csv'")

DataFrame 'df' saved to 'podcast_summary_results.csv'


In [ ]:
import pandas as pd

df = pd.read_csv("/content/podcast_summary_results.csv")

In [ ]:
# Generate sample data for 'chunk_summaries'
sample_chunk_summaries = [
    "This chunk discusses technology and AI innovations.",
    "Another chunk focuses on machine learning's impact on daily life.",
    "The final chunk considers ethical implications of AI deployment."
]

# Update the 'chunk_summaries' column in the DataFrame
df.at[0, 'chunk_summaries'] = sample_chunk_summaries

print("DataFrame 'df' updated with sample 'chunk_summaries' data.")
display(df)

DataFrame 'df' updated with sample 'chunk_summaries' data.


,transcript,chunk_summaries,final_summary
0,please subscribe our channel,[This chunk discusses technology and AI innova...,CNN.com will feature iReporter photos in a wee...


In [ ]:
# Generate a sample transcript
sample_transcript = "This is a sample podcast transcript. It contains various sentences about technology, artificial intelligence, and future innovations. We discuss new developments in machine learning and their impact on daily life. The conversation also touches upon ethical considerations in AI deployment."

# Update the 'transcript' column in the DataFrame
# Assuming df has at least one row, we update the first row's transcript
df.at[0, 'transcript'] = sample_transcript

print("DataFrame 'df' updated with sample transcript data.")
display(df)

DataFrame 'df' updated with sample transcript data.


,transcript,chunk_summaries,final_summary
0,This is a sample podcast transcript. It contai...,[This chunk discusses technology and AI innova...,CNN.com will feature iReporter photos in a wee...


In [ ]:
# Add 'Podcast_Name' and 'Episode_Title' columns with sample data
df['Podcast_Name'] = 'The Tech Daily'
df['Episode_Title'] = 'AI in Focus: Ethical Implications'

print("DataFrame 'df' updated with 'Podcast_Name' and 'Episode_Title' columns.")
display(df)

DataFrame 'df' updated with 'Podcast_Name' and 'Episode_Title' columns.


,transcript,chunk_summaries,final_summary,Podcast_Name,Episode_Title
0,This is a sample podcast transcript. It contai...,[This chunk discusses technology and AI innova...,CNN.com will feature iReporter photos in a wee...,The Tech Daily,AI in Focus: Ethical Implications


In [ ]:
print("DataFrame Info:")
df.info()

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   transcript       1 non-null      object
 1   chunk_summaries  1 non-null      object
 2   final_summary    1 non-null      object
 3   Podcast_Name     1 non-null      object
 4   Episode_Title    1 non-null      object
dtypes: object(5)
memory usage: 172.0+ bytes


In [ ]:
print("\nFirst 5 rows of DataFrame:")
display(df.head())


First 5 rows of DataFrame:


,transcript,chunk_summaries,final_summary,Podcast_Name,Episode_Title
0,This is a sample podcast transcript. It contai...,[This chunk discusses technology and AI innova...,CNN.com will feature iReporter photos in a wee...,The Tech Daily,AI in Focus: Ethical Implications


In [ ]:
print("\nFirst 5 rows of DataFrame:")
display(df.head())


First 5 rows of DataFrame:


,transcript,chunk_summaries,final_summary,Podcast_Name,Episode_Title
0,This is a sample podcast transcript. It contai...,[This chunk discusses technology and AI innova...,CNN.com will feature iReporter photos in a wee...,The Tech Daily,AI in Focus: Ethical Implications


In [ ]:
# Add 'Podcast_Name' and 'Episode_Title' columns with sample data
df['Podcast_Name'] = 'The Tech Daily'
df['Episode_Title'] = 'AI in Focus: Ethical Implications'

print("DataFrame 'df' updated with 'Podcast_Name' and 'Episode_Title' columns.")
display(df)

DataFrame 'df' updated with 'Podcast_Name' and 'Episode_Title' columns.


,transcript,chunk_summaries,final_summary,Podcast_Name,Episode_Title
0,This is a sample podcast transcript. It contai...,[This chunk discusses technology and AI innova...,CNN.com will feature iReporter photos in a wee...,The Tech Daily,AI in Focus: Ethical Implications


In [ ]:
print(df.columns)

Index(['transcript', 'chunk_summaries', 'final_summary', 'Podcast_Name',
       'Episode_Title'],
      dtype='object')


In [ ]:
display(df[['transcript', 'chunk_summaries', 'final_summary']])

,transcript,chunk_summaries,final_summary
0,This is a sample podcast transcript. It contai...,[This chunk discusses technology and AI innova...,CNN.com will feature iReporter photos in a wee...
